In [12]:
import numpy as np
import pandas as pd
import subprocess as sub
import matplotlib.pyplot as plt
import time

## step 1. Generate sample set by metropolis method

In [23]:
start = time.time()
sub.run(['g++', 'step1_plaquette_eff.cpp', '-o', 'step1_e', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib', '-ltrng4', '-std=gnu++11'])
sub.run(['./step1_e'])
print("step1 finished :", time.time()-start)

#Data of Energy&nn in fileout.txt
f = pd.read_csv('fileout_eff.txt', sep = ' ', header = 0)
temp = f.temp[0]
J0 = np.array(f.ene)
J1 = np.array(f.nn)
J2 = np.array(f.nnn)
J3 = np.array(f.nnnn)
num = len(J0)
nth = int(f.nth[0]) #How much consideration of nearest neighbors
Jmat = np.array([J1, J2, J3]).T
target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
print("T =", temp, " Energy, J =", Jlist, " error =", err)

#Data of coefficient in filein.txt
with open("filein_eff.txt", "w") as file:
    file.write('%d\n' %nth)
    file.write('%.10g\n' %temp)
    for i in range (nth + 1):
        data = Jlist[i]
        file.write('%.10g\n' % data)

step1 finished : 416.4611840248108
T = 4.493  Energy, J = [ 1.47651929  1.09807304 -0.02125913]  error = [31530.34719981]


## step 2. optimize J, regenerate sample set by wolff method

In [24]:
sub.run(['g++', 'step2_wolff_eff.cpp', '-o', 'step2_e', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib','-ltrng4', '-std=gnu++11'])
for i in range (15):
    start = time.time()
    sub.run(['./step2_e'])
    print("step2 :", time.time()-start)

    #Data of Energy&nn in fileout.txt
    f = pd.read_csv('fileout_eff.txt', sep = ' ', header = 0)
    temp = f.temp[0]
    J0 = np.array(f.ene)
    J1 = np.array(f.nn)
    J2 = np.array(f.nnn)
    J3 = np.array(f.nnnn)
    num = len(J0)
    nth = int(f.nth[0]) #How much consideration of nearest neighbors
    Jmat = np.array([J1, J2, J3]).T
    target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
    Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
    print("T =", temp, " Energy, J =", Jlist, " error =", err)
    #Data of coefficient in filein.txt
    with open("filein_eff.txt", "w") as file:
        file.write('%d\n' %nth)
        if temp>2.5: file.write('%.10g\n' %temp)
        else: file.write('%.10g\n' %(temp+0.2))
        for i in range (nth + 1):
            data = Jlist[i]
            file.write('%.10g\n' % data)

step2 : 287.380398273468
T = 4.293  Energy, J = [ 1.6410897   1.10307061 -0.02351462]  error = [30475.1930075]
step2 : 248.8516025543213
T = 4.093  Energy, J = [ 2.08377636  1.11311873 -0.02882472]  error = [28328.48602762]
step2 : 207.5419099330902
T = 3.8930000000000002  Energy, J = [ 2.19832035  1.11721537 -0.03104816]  error = [27815.46402257]
step2 : 169.26204800605774
T = 3.693  Energy, J = [ 2.66925769  1.12765039 -0.03655675]  error = [26308.37933666]
step2 : 140.03347659111023
T = 3.4930000000000003  Energy, J = [ 3.12341604  1.13554559 -0.03836167]  error = [24802.46732445]
step2 : 102.29126310348511
T = 3.293  Energy, J = [ 3.8434297   1.15027283 -0.04656599]  error = [22608.48356538]
step2 : 61.2556586265564
T = 3.093  Energy, J = [ 4.5755449   1.16286066 -0.0527649 ]  error = [19664.04461459]
step2 : 24.430070638656616
T = 2.8930000000000002  Energy, J = [ 6.11923318  1.19104173 -0.06990772]  error = [16326.98427709]
step2 : 12.228151082992554
T = 2.693  Energy, J = [ 7.32

## step 3. Compare magnetization

In [ ]:
start = time.time()
sub.run(['g++', 'step3_plot_eff.cpp', '-o', 'step3_e', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib', '-ltrng4', '-std=gnu++11'])
sub.run(['./step3_e'])
print("step3 :", time.time()-start)

In [ ]:
cols = ['m', 'ms', 'e', 'sp_h']
p = pd.read_csv('../../python/txtfiles-5/p10.txt', sep=' ', header=0)
fit = pd.read_csv('plot_eff.txt', sep = ' ', header = 1)

print("Plaquette & Fitted of L=10")

for j in range (4):
    plt.plot(p.temperature, p[cols[j]], 'r.', markersize = 10, markerfacecolor = None, linestyle = 'None', label = 'original')
    plt.plot(fit.temperature[fit.temperature>=1.5], fit[cols[j]][fit.temperature>=1.5], '.', markersize = 3, label = 'fitted')
    plt.xlabel('T', fontsize=14)
    plt.ylabel(cols[j], fontsize=14)
    plt.legend()
    plt.show()